In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

class CNN_LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_out_channels, kernel_size, output_size):
        super(CNN_LSTM_Model, self).__init__()
        
        # CNN layers
        self.cnn = nn.Conv1d(in_channels=input_size, 
                             out_channels=cnn_out_channels, 
                             kernel_size=kernel_size)
        
        # LSTM layers
        self.lstm = nn.LSTM(input_size=cnn_out_channels, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True)
        
        # Fully connected layer for output
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.cnn(x))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        
        return x

input_size = 175 

X = pd.read_csv('testX.csv')  
y = pd.read_csv('testY.csv')

y = y.replace({"Over": True, "Under": False})


X = X.to_numpy(dtype=np.float32)
y = y.to_numpy(dtype=np.float32)


X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1) 

sequence_length = 5  
num_samples = X.shape[0]
num_features = X.shape[1]
num_sequences = num_samples // sequence_length
X = X.reshape(num_sequences, num_features, sequence_length)
# y = y.view(-1, sequence_length)

X_data_permuted = X.permute(0, 2, 1) 

print(X_data_permuted.shape)

train_dataset = TensorDataset(X_data_permuted, y[:num_sequences])  
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


input_size = num_features 
hidden_size = 64
num_layers = 2
cnn_out_channels = 32
kernel_size = sequence_length
output_size = 1  
learning_rate = 0.001
num_epochs = 80

model = CNN_LSTM_Model(input_size, hidden_size, num_layers, cnn_out_channels, kernel_size, output_size)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), targets.squeeze())  

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Training complete!")

C:\Users\Twk23\AppData\Local\Temp\ipykernel_78388\1395382175.py:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({"Over": True, "Under": False})


torch.Size([175, 5, 19])
Epoch [1/80], Loss: 0.6941
Epoch [2/80], Loss: 0.6933
Epoch [3/80], Loss: 0.6913
Epoch [4/80], Loss: 0.6928
Epoch [5/80], Loss: 0.6901
Epoch [6/80], Loss: 0.6905
Epoch [7/80], Loss: 0.6896
Epoch [8/80], Loss: 0.6895
Epoch [9/80], Loss: 0.6883
Epoch [10/80], Loss: 0.6925
Epoch [11/80], Loss: 0.6907
Epoch [12/80], Loss: 0.6927
Epoch [13/80], Loss: 0.6880
Epoch [14/80], Loss: 0.6896
Epoch [15/80], Loss: 0.6914
Epoch [16/80], Loss: 0.6887
Epoch [17/80], Loss: 0.6906
Epoch [18/80], Loss: 0.6896
Epoch [19/80], Loss: 0.6904
Epoch [20/80], Loss: 0.6905
Epoch [21/80], Loss: 0.6898
Epoch [22/80], Loss: 0.6926
Epoch [23/80], Loss: 0.6897
Epoch [24/80], Loss: 0.6923
Epoch [25/80], Loss: 0.6922
Epoch [26/80], Loss: 0.6917
Epoch [27/80], Loss: 0.6905
Epoch [28/80], Loss: 0.6912
Epoch [29/80], Loss: 0.6898
Epoch [30/80], Loss: 0.6928
Epoch [31/80], Loss: 0.6914
Epoch [32/80], Loss: 0.6884
Epoch [33/80], Loss: 0.6890
Epoch [34/80], Loss: 0.6905
Epoch [35/80], Loss: 0.6935
Epoc

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

test_X = pd.read_csv('testX.csv')
test_y = pd.read_csv('testY.csv')

test_y = test_y.replace({"Over": 1, "Under": 0})

test_X = test_X.to_numpy(dtype=np.float32)
test_y = test_y.to_numpy(dtype=np.float32)

test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32).unsqueeze(1) 

sequence_length = 5  
num_samples = test_X.shape[0]
num_features = test_X.shape[1]
num_sequences = num_samples // sequence_length
test_X = test_X.reshape(num_sequences, num_features, sequence_length) 
test_X_data_permuted = test_X.permute(0, 2, 1)

test_dataset = TensorDataset(test_X_data_permuted, test_y[:num_sequences])  # Ensure y matches the number of sequences
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():  
    for inputs, targets in test_loader:

        outputs = model(inputs)
        
        targets = targets.squeeze()

        all_preds.extend(torch.sigmoid(outputs).squeeze().cpu().numpy())  
        all_labels.extend(targets.cpu().numpy()) 


all_preds_binary = [1 if pred > 0.5 else 0 for pred in all_preds]

accuracy = accuracy_score(all_labels, all_preds_binary)
print(f"Accuracy: {accuracy:.4f}")

conf_matrix = confusion_matrix(all_labels, all_preds_binary)
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.5371
Confusion Matrix:
[[94  0]
 [81  0]]


C:\Users\Twk23\AppData\Local\Temp\ipykernel_78388\1882857975.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_y = test_y.replace({"Over": 1, "Under": 0})


In [ ]:
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size):
        super(TransformerTimeSeriesModel, self).__init__()
        
        # Linear layer to project inputs to d_model dimensions for the Transformer
        self.input_fc = nn.Linear(input_size, d_model)
        
        # Transformer Encoder layer
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward),
            num_layers=num_encoder_layers
        )
        
        # Fully connected output layer
        self.output_fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.input_fc(x)
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)
        x = x[-1, :, :]
        x = self.output_fc(x)
        
        return x

# Parameters
input_size = 10  
d_model = 64    
nhead = 4     
num_encoder_layers = 2 
dim_feedforward = 128  
output_size = 1        

# Model instantiation
transformer_model = TransformerTimeSeriesModel(input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size)